In [1]:
import pandas as pd
df = pd.read_csv("./DATA/Tokyo house price.csv",encoding="cp932")

In [ ]:
# # 列名列表
# columns = ['種類', '価格情報区分', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称','最寄駅：距離（分）', '取引価格（総額）', '坪単価', '面積（㎡）', '取引価格（㎡単価）', '土地の形状', '間口','延床面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類','前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時期', '取引の事情等']

# # 打印每一列的唯一值和唯一值的数量
# for column in columns:
#     unique_values = df[column].unique()
#     unique_count = df[column].nunique()
#     print(f"{column}: 有 {unique_count} 个唯一值。")
#     print(f"这些唯一值是: {unique_values}")
#     print()

In [2]:
#清空取引の事情等里面有内容的
filtered_df = df[df['取引の事情等'].isna()]

In [3]:
# 筛选出'取引時期'列中包含2018年至2023年的数据
df['取引時期'] = filtered_df ['取引時期'].astype(str)
filtered_df = filtered_df [filtered_df ['取引時期'].str.contains(r'202[0-3]|201[8-9]', regex=True)]
filtered_df = filtered_df .query('種類 == "宅地(土地と建物)"')
filtered_df = filtered_df .query('種類 == "宅地(土地と建物)"')
filtered_df = filtered_df .query('今後の利用目的 == "住宅"')

In [4]:
#使用这些做特征并清除所有空值
selected_columns = ['最寄駅：距離（分）', '取引価格（総額）', '面積（㎡）','建築年', '建物の構造','地区名',"建ぺい率（％）","容積率（％）"]
selected_df = filtered_df[selected_columns]
selected_df = selected_df.dropna()

In [5]:

# 提取建筑年份的年份部分并转换为整数
selected_df['建築年'] = selected_df['建築年'].str.extract('(\d+)')
selected_df = selected_df.dropna()
selected_df['建築年'] = selected_df['建築年'].astype(int)


In [ ]:
selected_df.info()

In [6]:
#计算建筑年数
import pandas as pd
from datetime import datetime
current_year = datetime.now().year
selected_df['建築年数'] = current_year - selected_df['建築年']
selected_df = selected_df.drop("建築年",axis=1)

In [7]:
#使用encoder编码建物の構造
from sklearn.preprocessing import LabelEncoder
# 确保列为字符串类型
for column in [ '建物の構造']:
    selected_df[column] = selected_df[column].astype(str)

# 创建LabelEncoder对象
label_encoder = LabelEncoder()

# 对每个object列进行标签编码
for column in [ '建物の構造']:
    selected_df[column] = label_encoder.fit_transform(selected_df[column])

print(selected_df)

       最寄駅：距離（分）    取引価格（総額） 面積（㎡）  建物の構造   地区名  建ぺい率（％）  容積率（％）  建築年数
0              4   340000000   100      7   飯田橋     80.0   500.0    17
22             2  4000000000   770     13   一番町     80.0   600.0    48
29             4   210000000   135      1   岩本町     80.0   700.0    78
45             3   100000000    45      1   岩本町     80.0   600.0     3
50             4  1500000000   220      7   岩本町     80.0   600.0     3
...          ...         ...   ...    ...   ...      ...     ...   ...
302303        25    22000000   155      1  大字平井     40.0    80.0     5
302304        21    20000000   115      1  大字平井     40.0    80.0     6
302310        16    29000000   165      1  大字平井     40.0    80.0     7
302312        13     9400000   100      1  大字平井     40.0    80.0    19
302812         9     7800000   410      1    梅澤     40.0    80.0    48

[43572 rows x 8 columns]


In [ ]:
selected_df = selected_df.dropna(subset=['最寄駅：距離（分）'])

In [9]:
print(selected_df['地区名'].isna().sum())

0


In [10]:
#使用one-hot编码地区
from sklearn.preprocessing import OneHotEncoder
# 使用OneHotEncoder进行独热编码
# 使用 OneHotEncoder 进行独热编码
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_features = one_hot_encoder.fit_transform(selected_df[['地区名']])

# 获取独热编码后的特征名
feature_names = one_hot_encoder.get_feature_names_out(['地区名'])

# 将编码后的特征转换为 DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=feature_names, index=selected_df.index)

# 确认编码后的 DataFrame 中没有空值
print(encoded_df.isna().sum().sum())

# 合并原始数据和独热编码后的数据
df_onehot = pd.concat([selected_df, encoded_df], axis=1)

# 删除原始的 '地区名' 列
df_onehot = df_onehot.drop("地区名", axis=1)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0


In [ ]:
print(feature_names)

In [ ]:
df_onehot['最寄駅：距離（分）'].unique()

In [11]:
# 定义映射关系
distance_mapping = {
    '30分～60分': 30,
    '1H～1H30': 60,
    '1H30～2H': 90,
    '2H～': 120,
    '2,000㎡以上': 120
}

# 将文本值映射成数字
df_onehot['最寄駅：距離（分）'] = df_onehot['最寄駅：距離（分）'].map(lambda x: distance_mapping[x] if x in distance_mapping else int(x))

In [12]:
# 定义映射关系
distance_mapping = {
    '2,000㎡以上': 2000
}
# 将文本值映射成数字
df_onehot['面積（㎡）'] = df_onehot['面積（㎡）'].map(lambda x: distance_mapping[x] if x in distance_mapping else int(x))

In [13]:
# 定义一个函数来去除离群值
# 四分位距（IQR）方法是一种常用的检测离群值的方法。它利用数据的第一四分位数（Q1）和第三四分位数（Q3）来定义一个范围，任何超出该范围的值都被视为离群值。
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

# 去除 '取引価格（総額）' 的离群值
data_cleaned = remove_outliers(df_onehot, '取引価格（総額）')

In [14]:
import numpy as np
# 定义一个函数来去除离群值
# Z-Score 方法是通过计算每个数据点与均值的标准差来检测离群值的。通常，Z-Score 大于 3 或小于 -3 的数据点被视为离群值。
def remove_outliers_zscore(df, column):
    mean = df[column].mean()
    std = df[column].std()
    z_scores = (df[column] - mean) / std
    df_filtered = df[np.abs(z_scores) < 3]
    return df_filtered

# 去除 '取引価格（総額）' 的离群值
data_cleaned = remove_outliers_zscore(data_cleaned, '取引価格（総額）')

In [15]:
filtered_data = data_cleaned[(data_cleaned['取引価格（総額）'] > 5000000) & (data_cleaned['取引価格（総額）'] <= 100000000)]
filtered_data = filtered_data[filtered_data['面積（㎡）'] < 200]
filtered_data = filtered_data[filtered_data['最寄駅：距離（分）'] < 30]


In [ ]:
filtered_data.info()

In [16]:
filtered_data.to_csv('./DATA/exported_data.csv', index=False, encoding='utf-8')